In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the Excel file
df = pd.read_excel("PostsByUsers.xlsx")

# Keep only rows with valid post_date values
df = df[df['post_date'].notna()].copy()

# Convert post_date to datetime, remove timezone information
df['post_date'] = pd.to_datetime(df['post_date'], utc=True).dt.tz_convert(None)

# Group by user and compute tweet statistics
user_stats = df.groupby(['user_id', 'user_name']).agg(
    num_posts=('tweet_id', 'count'),
    first_post_date=('post_date', 'min'),
    last_post_date=('post_date', 'max')
).reset_index()

# Calculate the time span and tweet frequency per day
user_stats['days_span'] = (user_stats['last_post_date'] - user_stats['first_post_date']).dt.days + 1
user_stats['tweets_per_day'] = user_stats['num_posts'] / user_stats['days_span']

# Classify activity level: 
# L = 0-1 tweets/day, M = 2-5 tweets/day, H = >5 tweets/day
def classify_activity(tweets_per_day):
    if tweets_per_day <= 1:
        return 'L'
    elif tweets_per_day <= 5:
        return 'M'
    else:
        return 'H'

user_stats['activity_level'] = user_stats['tweets_per_day'].apply(classify_activity)

# Save to Excel
user_stats.to_excel("User_Tweet_Activity_Final_HML.xlsx", index=False)

# Count number of users in each activity level
activity_counts = user_stats['activity_level'].value_counts().reindex(['L', 'M', 'H'], fill_value=0)

# Plot bar chart
plt.figure(figsize=(6, 4))
activity_counts.plot(kind='bar', color=['skyblue', 'orange', 'lightgreen'])
plt.title("User Tweet Activity Levels", fontsize=14)
plt.xlabel("Activity Level", fontsize=12)
plt.ylabel("Number of Users", fontsize=12)
plt.xticks(ticks=[0, 1, 2], labels=['Low (0-1)', 'Medium (2-5)', 'High (>5)'], rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
